In [1]:
!pip install pandas
!pip install torch
!pip install torchvision
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install scikit-learn
!pip install tensorboard


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Hec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Hec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Hec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Hec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Cloning https://github.com/openai/CLIP.git to c:\users\hec\appdata\local\temp\pip-req-build-j3xl6vew
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Hec\AppData\Local\Temp\pip-req-build-j3xl6vew'

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Hec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Hec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\Hec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import csv
import glob
import numpy as np
import os
import pandas as pd
import pickle
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from pathlib import Path
from tqdm.auto import trange

from classes.adaptation import calc_loss, construct_model
from classes.asymmetricRecall import AsymmetricRecall
from classes.embDataset import EmbDataset

from utils.adaptationPreprocess import create_paired_embeddings_dict, merge_dicts_with_csv
from utils.train_functions import train, validate, split_train_test

c:\Users\Hec\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LEFT_FILES_PATH = 'data/train/left/*'
RIGHT_FILES_PATH = 'data/train/right/*'
ALL_FILES_PATH = 'data/train/all/*'
ALL_ENCODINGS_PATH = 'data/train_all_encodings.npy'
ALL_FULL_ENCODINGS_PATH = 'data/train_all_full_encodings.npy'
CSV_FILENAME = 'data/train.csv'
SAVE_TO = 'output/'

TEST_LEFT_FILES_PATH = 'data/test/left/*'
TEST_RIGHT_FILES_PATH = 'data/test/right/*'
TEST_ALL_FILES_PATH = 'data/test/all/*'
TEST_ALL_FULL_ENCODINGS_PATH = 'data/test_all_full_encodings.npy'
TEST_CSV_FILENAME = 'data/test_candidates.csv'

train_csv = 'data/train.csv'
train_candidates_csv = 'data/train_candidates.csv'
test_candidates_csv = 'data/test_candidates.csv'

In [3]:
is_cuda = torch.cuda.is_available()
if not is_cuda:
    device = torch.device("cpu")
else:
    device = torch.device("cuda")

Define parameters

In [27]:
runs = 1
temperature = 1
topk = [i for i in range(1, 21)]
epochs = 20
test_split = 0.8
model_name = 'original'

Import peviously computed encodings

In [28]:
all_encodings = np.load(ALL_ENCODINGS_PATH)
all_full_encodings = np.load(ALL_FULL_ENCODINGS_PATH)

dimensions_before_pca = all_encodings.shape[1]
print('All encodings shape: ', all_encodings.shape)
print('All full encodings shape:', all_full_encodings.shape)

All encodings shape:  (4000, 640)
All full encodings shape: (4000, 5440)


Match embeddings with their original image name, link the left image with the ground truth right image

In [52]:
# Start with training data for validation score
images = np.load('data/train_all_full_encodings.npy')
pca = pickle.load(open('models/pca_module.pkl', 'rb'))
# Load encoded images
train_images_enc = np.load('data/train_all_full_encodings.npy')
test_images_enc = np.load('data/test_all_full_encodings.npy')

# Read candidates
test_candidates = pd.read_csv(test_candidates_csv)
train_candidates = pd.read_csv(train_candidates_csv)

# Get images names
test_images_names = np.array([x.split('.')[0] for x in os.listdir('data/test/all')])
train_images_names = np.array([x.split('.')[0] for x in os.listdir('data/train/all')])

# Apply PCA on embedded images
images_train_pca = pca.transform(train_images_enc)
images_test_pca = pca.transform(test_images_enc)

# # Get dataframe with cosine similarities between left image and candidates
# df_train = cos_sim_enc(images_train_pca, train_images_names, train_candidates)
# df_test = cos_sim_enc(images_test_pca, test_images_names, test_candidates)

# # Make the values into probabilities with softmax
# df_train_as_probs = make_df_as_probs(df_train)
# df_test_as_probs = make_df_as_probs(df_test)

# # Evaluate performance on training set
# train_labels = pd.read_csv('data/train.csv')
# eval(df_train_as_probs, train_labels, train_candidates)

# # Write csv
# df_test_as_probs.to_csv('output/cos_sim_full_enc_pca.csv', index=False)

c:\Users\Hec\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator PCA from version 1.3.0 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [53]:
paired_embeddings = create_paired_embeddings_dict(images_train_pca, ALL_FILES_PATH, LEFT_FILES_PATH, RIGHT_FILES_PATH)

merged_dicts = merge_dicts_with_csv(CSV_FILENAME, paired_embeddings)
print(merged_dicts['left'].shape)

(2000, 256)


Define et construct the model

In [54]:
model = construct_model(model_name, 256).to(device)

Load the dataset and split

In [55]:
embds_dataset = EmbDataset(merged_dicts, only_original=False)

print("Total dataset size:  ", len(embds_dataset))
in_dim = embds_dataset[0][0]["left"].shape[0]

splitted = split_train_test(embds_dataset, test_split, device)
train_set, valid_set = splitted["train"], splitted["test"]

print("train set without aug size ", len(train_set[0]["left"]))
print("train set with    aug size ", len(train_set[1]["left"]))
print("test  set without aug size  ", len(valid_set["left"]))

metrics_of_all_runs = []

info = (
    "Embd path: "
    + str(ALL_FULL_ENCODINGS_PATH)
    + "\n"
    + "Using augmented images: "
    + str(embds_dataset.augmented)
    + "\n"
)

Total dataset size:   2000
train set without aug size  200
train set with    aug size  200
test  set without aug size   800


Train the model, save it, and get the metrics/loss for each run

In [56]:
for run in trange(runs):
    run_path = Path(SAVE_TO) / f"run{run+1}"
    model = construct_model(model_name, in_dim).to(device)

    writer = SummaryWriter(run_path)
    writer.add_text("Temperature: ", str(temperature))
    writer.add_text("Model", str(model).replace("\n", "  \n"))
    writer.add_text("Informations: ", info.replace("\n", "  \n"))

    if model != "dummy":
        optimizer = torch.optim.Adam(model.parameters())
        # writer.add_text("Optimizer", str(optimizer).replace("\n", "  \n"))

        metrics_per_run = train(
            model,
            optimizer,
            calc_loss,
            train_set,
            valid_set,
            epochs,
            run_path,
            temperature,
            topk,
            device,
        )
        # save the model and the optimizer state_dics for this run
        torch.save(
            {
                "epoch": epochs,
                "state_dict": model.state_dict(),
                "optimzier": optimizer.state_dict(),
            },
            run_path / "model_and_optimizer.pt",
        )
    else:
        metrics_per_run = validate(
            model,
            calc_loss,
            valid_set,
            temperature,
            topk,
            device,
        )
        for metric, val in metrics_per_run.items():
            writer.add_scalar(metric, val, 1)
            writer.flush()
            

    np.save(run_path / "data", metrics_per_run)

    metrics_of_all_runs.append(metrics_per_run)

    splitted = split_train_test(embds_dataset, test_split, device)
    train_set, valid_set = splitted["train"], splitted["test"]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:06<00:00,  6.45s/it]


Save all the metrics

In [57]:
# save all runs metrics into one file
np.save(Path(SAVE_TO) / "all_runs", metrics_of_all_runs)

avg_path = Path(SAVE_TO) / "avg"
writer = SummaryWriter(avg_path)

Display the saved metrics

In [58]:
test = np.load('output/run1/data.npy', allow_pickle=True)
print(test)

{'train': {'Train/Loss': [5.124790191650391, 5.104914665222168, 5.085371017456055, 5.066212177276611, 5.047489166259766, 5.029247283935547, 5.011515140533447, 4.994318962097168, 4.977666854858398, 4.961581230163574, 4.946057319641113, 4.931093215942383, 4.916676998138428, 4.902804851531982, 4.889459609985352, 4.876623153686523, 4.8642706871032715, 4.852385997772217, 4.840950012207031, 4.829952239990234], 'Train/top 1': [0.35, 0.395, 0.435, 0.495, 0.545, 0.6, 0.63, 0.675, 0.7, 0.735, 0.76, 0.805, 0.83, 0.835, 0.86, 0.875, 0.885, 0.91, 0.92, 0.93], 'Train/top 2': [0.43, 0.46, 0.535, 0.58, 0.615, 0.685, 0.705, 0.74, 0.775, 0.83, 0.865, 0.885, 0.905, 0.905, 0.91, 0.93, 0.945, 0.96, 0.96, 0.96], 'Train/top 3': [0.47, 0.545, 0.585, 0.63, 0.67, 0.73, 0.75, 0.79, 0.84, 0.865, 0.885, 0.91, 0.92, 0.925, 0.935, 0.95, 0.965, 0.97, 0.975, 0.98], 'Train/top 4': [0.495, 0.555, 0.61, 0.66, 0.705, 0.755, 0.79, 0.83, 0.87, 0.885, 0.905, 0.915, 0.93, 0.935, 0.945, 0.955, 0.965, 0.975, 0.985, 0.99], 'Trai

### Inference

We load the test_dataset, and the previously saved model. We get the prediction of the model on the test dataset, then post process it before submitting

In [59]:
from sklearn.metrics.pairwise import cosine_similarity

def cos_sim_np(left, right_images):
    """Compute cosine similarity between left image and all right images in row"""
    return np.array([cosine_similarity(left, right_images[i])[0][0] for i in range(len(right_images))])

def cos_sim_enc(images_enc, images_names, candidates_csv):
    """returns a data frame with format suitable for submitting, except that values 
    don't represent probabilities but cosine similarities."""
    results = []
    start_time = time.time()
    for i, row in candidates_csv.iterrows():

        # Compute cosine similarity between left and all other images in row
        left = images_enc[np.where(images_names == row['left'])]
        right_images = np.array([images_enc[np.where(images_names == row[f'c{i}'])] for i in range(20)])
        sim_array = cos_sim_np(left, right_images)
        res = [row['left']] + list(sim_array)
        results.append(list(res))

        if i%400 == 0 and i!=0:
            print(f"Processed: {i}")
            print("Elapsed time: ", time.time() - start_time)
            start_time = time.time()
            
    results = np.array(results)
    column_names = ['left'] + [f'c{i}' for i in range(20)]
    df = pd.DataFrame(results, columns=column_names)

    return df

# Functions to make dataframes values into probabilities
def softmax(x):
    """Compute softmax values for each sets of scores in x (usually a row of dataframe)."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def make_df_as_probs(df):
    # Make the values into probabilities with softmax
    df_as_probs = df.apply(lambda row: softmax(np.array(row[1:]).astype(np.float64)), axis=1)
    # Make into new dataframe
    df_as_probs = pd.DataFrame(df_as_probs.values.tolist(), columns=df.columns[1:])
    # Add left column in first position
    df_as_probs.insert(0, 'left', df['left'])
    
    return df_as_probs

# Evaluate result for training data (gives an idea of performance). For each row in df_train get 
# index of colmuns of top 2 values and check if the true match is in the top 2
def get_score(row, row_nb, true_labels, candidates):
    """return 1 if true label in top_2, else return 0"""
    # Get top 2
    top_2 = row[1:].argsort()[-2:][::-1].values

    # Get true label
    label_row = true_labels[true_labels['left'] == row['left']]
    true_label = label_row['right'].values[0]

    # Get top 2 predicted labels. i+1 to account for 'left' column
    top_2_names = [candidates.iloc[row_nb, i+1] for i in top_2]
    if true_label in top_2_names:
        return 1
    else:
        return 0

def eval(df, true_labels, candidates):
    """Evaluate score on df"""
    score = 0 
    for i, row in df.iterrows():
        score += get_score(row, i, true_labels, candidates)
    return score/len(df)

In [60]:
print(type(model))
model.eval()

# output = model()

<class 'classes.adaptation.Adaptation'>


Adaptation(
  (weight_matrix): Linear(in_features=256, out_features=1024, bias=False)
)

In [61]:
def new_sim_matrix(a, b, eps=1e-8):
    """
    Calculate the similarity matrix given two lists of embeddings.
    added eps for numerical stability.
    """
    a_n, b_n = a.norm(dim=1)[:, None], b.norm(dim=1)[:, None]
    a_norm = a / torch.max(a_n, eps * torch.ones_like(a_n))
    b_norm = b / torch.max(b_n, eps * torch.ones_like(b_n))
    sim_mt = torch.mm(a_norm, b_norm.transpose(0, 1))

    return sim_mt

In [62]:
def inference_loss(left, right, temp, device):
    """
    Our loss function that is used in training.
    """
    sim1 = new_sim_matrix(left, right)
    sim2 = sim1.t()

    num_classes = 20
    one_hot_encoded_list = torch.tensor(np.array([np.eye(num_classes)[i - 1] for i in range(1, num_classes + 1)]))
    # print(torch.arange(len(sim1)).long().to(device))
    loss_left2right = F.cross_entropy(
        sim1 * temp, one_hot_encoded_list
    )
    loss_right2left = F.cross_entropy(
        sim2 * temp, one_hot_encoded_list
    )
    loss = loss_left2right * 0.5 + loss_right2left * 0.5

    return loss

In [63]:
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity


class NewAsymmetricRecall:
    """
    This class is responsible for calculating the topk matches to our query.
    """
    def __init__(self, left_ebds, right_ebds):

        self.M = cosine_similarity(left_ebds, right_ebds)
        self.sorted_idx_1 = self.M.argsort(1)
        self.sorted_idx_2 = self.M.T.argsort(1)

    def eval(self, at):
        # print(self.sorted_idx_1)
        sens1 = np.array(
            [lbl in self.sorted_idx_1[lbl][:at] for lbl in range(len(self.M))]
        )
        sens2 = np.array(
            [lbl in self.sorted_idx_2[lbl][:at] for lbl in range(len(self.M))]
        )

    def get_scores(self):
        return self.M

In [75]:
def inference(model, loss_func, topk, left, right_images):
    """Compute cosine similarity between left image and all right images in row"""
    with torch.no_grad():
        left_tensor = torch.tensor(np.array([left for i in range(20)]))
        right_tensor = torch.tensor(right_images)
        out_left, out_right = model(left_tensor.view(20,256)), model(right_tensor.view(20,256))

        test_loss = loss_func(out_left, out_right, temperature, device)
        # print(test_loss)
        loss_dict = {"Test/Loss": test_loss.item()}

        aR = NewAsymmetricRecall(out_left.detach().cpu(), out_right.detach().cpu())
        scores = aR.get_scores()
        # for k in topk:
        #     metrics[f"Test/top {k}"] = aR.eval(at=k)
        return scores
    # return np.array([model(left, right_images[i])[0][0] for i in range(len(right_images))])


def get_score(model, loss_func, topk,images_enc, images_names, candidates_csv):
    """returns a data frame with format suitable for submitting, except that values 
    don't represent probabilities but cosine similarities."""
    results = []
    start_time = time.time()
    for i, row in candidates_csv.iterrows():

        # Compute cosine similarity between left and all other images in row
        # print(np.where(images_names == row['left'])[0] -1)
        # print([np.where(images_names == row[f'c{i}']) for i in range(20)])
        left = images_enc[np.where(images_names == row['left'])[0] -1]
        right_images = np.array([images_enc[np.where(images_names == row[f'c{i}'])] for i in range(20)])
        sim_array = inference(model, loss_func, topk, left, right_images)
        # print(sim_array.values())
        res = [row['left']] + list(sim_array.tolist())
        results.append(list(res))

        if i%400 == 0 and i!=0:
            print(f"Processed: {i}")
            print("Elapsed time: ", time.time() - start_time)
            start_time = time.time()
        # break
    results = np.array(results)
    column_names = ['left'] + [f'c{i}' for i in range(20)]
    df = pd.DataFrame(results, columns=column_names)

    return df

In [77]:
print(images_test_pca.shape, type(images_test_pca), )
df = get_score(model, inference_loss, topk, images_test_pca, test_images_names, test_candidates)

(4000, 256) <class 'numpy.ndarray'>


RuntimeError: expected scalar type Double but found Float

In [72]:
def extract_i(tup, i):
        return tup[i]
output_df = df.copy()
# Iterate from i = 1 to 20 and create new columns
for i in range(20):
    output_df[f'c{i}'] = output_df.apply(lambda row: extract_i(row[f'c{i}'], i), axis=1)





In [46]:
output_df

,left,c0,c1,c2,c3,c4,c5,c6,c7,c8,...,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19
0,abm,0.181642,0.113814,0.000000,0.449350,0.463243,0.140958,0.058896,0.295562,0.000000,...,0.594338,0.000000,0.164508,0.000000,0.000000,0.000000,0.000000,0.421807,0.000000,0.370343
1,aci,0.000000,0.445127,0.025418,0.433957,0.107833,0.040855,0.127680,0.000000,0.000000,...,0.551866,0.000000,0.125805,0.063414,0.000000,0.000000,0.311907,0.146539,0.064012,0.083413
2,acn,0.267696,0.331837,0.483278,0.365026,0.499440,0.000000,0.302881,0.000000,0.406361,...,0.022382,0.415780,0.164252,0.082173,0.052361,0.058619,0.000000,0.000000,0.206016,0.170278
3,aco,0.000000,0.072751,0.000000,0.234525,0.096996,0.618933,0.404463,0.344928,0.477846,...,0.205659,0.071408,0.008396,0.000000,0.616967,0.084023,0.152752,0.196317,0.384676,0.508242
4,acu,0.409283,0.000000,0.237088,0.000000,0.000000,0.000000,0.269154,0.394364,0.373098,...,0.000000,0.198789,0.000000,0.187270,0.511895,0.000000,0.000051,0.001442,0.208351,0.542109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,zyc,0.000000,0.182884,0.280631,0.454810,0.292498,0.000000,0.381388,0.163035,0.000000,...,0.000000,0.196437,0.639109,0.102266,0.122826,0.302260,0.242143,0.061088,0.096414,0.000000
1996,zyi,0.000000,0.000000,0.103475,0.028997,0.004914,0.566185,0.224597,0.003370,0.019888,...,0.000279,0.000000,0.068159,0.348148,0.145478,0.076007,0.047732,0.376220,0.887204,0.000000
1997,zym,0.000000,0.599683,0.060244,0.160129,0.000000,0.006686,0.145412,0.065172,0.309074,...,0.243469,0.056420,0.000000,0.580711,0.156169,0.000000,0.084407,0.257787,0.402024,0.000000
1998,zzq,0.009501,0.110197,0.001117,0.050613,0.750223,0.400474,0.739369,0.709926,0.638798,...,0.476169,0.102078,0.292928,0.010006,0.735017,0.062852,0.156087,0.359236,0.068904,0.112887


In [47]:
probas_output_df = make_df_as_probs(output_df)

In [50]:
probas_output_df

,left,c0,c1,c2,c3,c4,c5,c6,c7,c8,...,c10,c11,c12,c13,c14,c15,c16,c17,c18,c19
0,abm,0.049975,0.046698,0.041674,0.065315,0.066229,0.047983,0.044202,0.056005,0.041674,...,0.075506,0.041674,0.049126,0.041674,0.041674,0.041674,0.041674,0.063541,0.041674,0.060354
1,aci,0.042491,0.066315,0.043585,0.065579,0.047329,0.044263,0.048278,0.042491,0.042491,...,0.073785,0.042491,0.048187,0.045273,0.042491,0.042491,0.058044,0.049197,0.045300,0.046187
2,acn,0.053136,0.056656,0.065920,0.058568,0.066994,0.040657,0.055039,0.040657,0.061040,...,0.041577,0.061617,0.047914,0.044139,0.042842,0.043111,0.040657,0.040657,0.049958,0.048204
3,aco,0.038340,0.041233,0.038340,0.048474,0.042245,0.071196,0.057453,0.054132,0.061827,...,0.047095,0.041178,0.038663,0.038340,0.071056,0.041701,0.044668,0.046657,0.056327,0.063735
4,acu,0.062461,0.041482,0.052581,0.041482,0.041482,0.041482,0.054294,0.061536,0.060242,...,0.041482,0.050605,0.041482,0.050026,0.069211,0.041482,0.041484,0.041542,0.051091,0.071334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,zyc,0.040928,0.049141,0.054187,0.064498,0.054834,0.040928,0.059932,0.048176,0.040928,...,0.040928,0.049812,0.077550,0.045335,0.046277,0.055372,0.052141,0.043506,0.045071,0.040928
1996,zyi,0.041815,0.041815,0.046373,0.043045,0.042021,0.073658,0.052344,0.041956,0.042655,...,0.041826,0.041815,0.044764,0.059228,0.048363,0.045117,0.043859,0.060914,0.101540,0.041815
1997,zym,0.041958,0.076428,0.044564,0.049245,0.041958,0.042240,0.048525,0.044784,0.057154,...,0.053525,0.044393,0.041958,0.074992,0.049050,0.041958,0.045653,0.054296,0.062721,0.041958
1998,zzq,0.036013,0.039829,0.035713,0.037525,0.075536,0.053243,0.074721,0.072553,0.067571,...,0.057429,0.039506,0.047814,0.036031,0.074396,0.037987,0.041699,0.051092,0.038217,0.039936


In [51]:
probas_output_df.to_csv('output/adaptative.csv', index=False)